# Data Analysis
Visualize the data where we use.

In [ ]:
import pandas as pd
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
from thesispack.methods import position_figure, timestamp_figure, timestamp_grad_figure
from thesispack.methods import splited_timestamp_figure, tod_and_spectrum_per_ap_with_pad
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [ ]:
df=pd.read_csv(
    StringIO(
        str(
            np.load("../data/Data.zip")['RTT_data.csv'],
            'utf-8'
    )))

## Map of Points

In [ ]:
ap_points_with_index = df[["AP_index", "AP_positionX[m]", "AP_positionY[m]"]].drop_duplicates()
ap_points_with_index = ap_points_with_index.to_numpy()
ap_points_with_indexdf=pd.read_csv(
    StringIO(
        str(
            np.load("../data/Data.zip")['RTT_data.csv'],
            'utf-8'
    )))


In [ ]:
save_obj = ['.', 'data','data_pos']
position_figure(df, ap_points_with_index, None)


## Timestamp [split per experiment]

In [ ]:
data = df['%Timestamp[s]']
grad_data = np.gradient(data)
up_thres = -750
new_m_ind = [0] + list(np.where(grad_data < up_thres)[0][::2]) + [len(grad_data)]

In [ ]:
save_obj = ['.', 'data','timestamp']
timestamp_figure(data, r'$timestamp \, in \, sec.$', save_obj)

save_obj = ['.', 'data','grad_timestamp']
timestamp_grad_figure(grad_data, r'$timestamp \, in \, sec. \, (gradient)$', up_thres, save_obj)

save_obj = ['.', 'data','split_timestamp']
splited_timestamp_figure(data, new_m_ind, r'$timestamp \, in \, sec.$', save_obj)

## ToD per AP (sort term 5 sec.) with padding.

In [ ]:
dt = 5
tods_ap, _ =  tod_and_spectrum_per_ap_with_pad(df, new_m_ind, dt)

In [ ]:
tod_min = np.min(tods_ap)
tod_max = np.max(tods_ap)

In [ ]:
# 100, 200, 601, 500
expl_idx = 100

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 10))
im = ax.imshow(tods_ap[expl_idx], vmin=tod_min, vmax=tod_max, cmap='coolwarm')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
cbar = fig.colorbar(im, ax=ax,  orientation='horizontal')
cbar.ax.tick_params(labelsize=16)
cbar.set_label(label="ToD factor (m)", fontsize=20)
plt.xlabel(r'$\# \, of \, AP$', fontsize=20)
plt.ylabel(r'$\# \, of \, samples$', fontsize=20)

save_obj = ['.', 'data', 'tod', expl_idx]
# plt.savefig('{}/DataFigures/{}/{}-expl-{}.svg'.format(*save_obj))

## Spectrogram per AP (200 first samples).

In [ ]:
spct_df = df.iloc[:, 8: 57+11]
forminmax = spct_df.iloc[:, 3:].applymap(lambda x: np.abs(complex(x.replace('i', 'j')))).to_numpy()
forminmax = 20*np.log10(forminmax + 1)
sp_min = np.min(forminmax)
sp_max = np.max(forminmax)

In [ ]:
ap_idx = 1
split_idx = 0
split_df = spct_df.loc[new_m_ind[split_idx]+1:new_m_ind[split_idx+1]]
split_df = split_df[split_df['AP_index'] == ap_idx]
spectrumgram = split_df.iloc[:, 3: ].applymap(lambda x: np.abs(complex(x.replace('i', 'j')))).to_numpy()
spectrumgram = 20*np.log10(spectrumgram + 1)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,8))
im = ax.imshow(spectrumgram[:200].T, vmax=sp_max, vmin=sp_min, cmap='inferno')
# plt.tick_params(fontsize=16, axis = 'both')
plt.xticks(fontsize=16)
plt.yticks(ticks =np.linspace(1, 56, 5), labels=312.5*np.linspace(-58, -2, 5).astype(int), fontsize=16)
plt.xlabel(r'$\# \, of \, samples$', fontsize=20)
plt.ylabel(r'$Hz$', fontsize=20)

cbar = fig.colorbar(im, ax=ax,  orientation='horizontal')
cbar.ax.tick_params(labelsize=16)
cbar.set_label(label=r'$dBm$', fontsize=20)

save_obj = ['.', 'data', 'spectrumgram', ap_idx, split_idx]
# plt.savefig('{}/DataFigures/{}/{}-ap-{}-data_split-{}.svg'.format(*save_obj))

## Spectrogram per AP (sort term 5 sec.) with padding.

In [ ]:
dt = 5
_, pad_spectrumgrams =  tod_and_spectrum_per_ap_with_pad(df, new_m_ind, dt)
pad_spectrumgrams = 20*np.log10(pad_spectrumgrams + 1)

sps_min = np.min(pad_spectrumgrams)
sps_max = np.max(pad_spectrumgrams)


In [ ]:
example_idx = 0
ap_idx = 0
fig, ax = plt.subplots(1,1, figsize=(16,8))
im = ax.imshow(pad_spectrumgrams[example_idx,:, ap_idx], cmap='inferno', vmin=sps_min, vmax=sps_max)
plt.yticks(fontsize=16)
plt.xticks(ticks =np.linspace(1, 56, 5), labels=312.5*np.linspace(-58, -2, 5).astype(int), fontsize=16)

plt.ylabel(r'$\# \, of \, samples$', fontsize=20)
plt.xlabel(r'$Hz$', fontsize=20)

cbar = fig.colorbar(im, ax=ax,  orientation='horizontal')
cbar.ax.tick_params(labelsize=16)
cbar.set_label(label=r'$dBm$', fontsize=20)

save_obj = ['.', 'data', 'spectrumgram_pad', example_idx, ap_idx+1]
# plt.savefig('{}/DataFigures/{}/{}-expl-{}-ap-{}.svg'.format(*save_obj))